In [60]:
### Importing Libraries

In [45]:
import requests
import pandas as pd
import numpy as np
from pandasql import sqldf

import json
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import folium

# Integrating FourSquare API

In [46]:
CLIENT_ID = 'LZXKKR3DROVZZCAMQWVB0EVBRJH5URYIPI3EE5V54KHNGUAF' 
CLIENT_SECRET = 'SFMKJVVYDBZJYOLJOWM0FY0MSTOLAKAPYGDHT4UMXBVELY4K'
VERSION = '20200402'
LIMIT = 100

address = "Seoul, Korea"

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude =location.latitude
longitude =location.longitude
print('Location : {},{}'.format(latitude,longitude))

Location : 37.5666791,126.9782914


### **At 1KM checking the list of venues**

In [63]:
search_query_list = ['park','temple','church','shop','university','store'
                    'office','company']
radius =1000
temp_df=[]
search_df=[]

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

for i in range(len(search_query_list)):
    url= 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query_list[i], radius, LIMIT)
    results=requests.get(url).json()
    venues = results['response']['venues']
    dataframe = json_normalize(venues)
    dataframe.columns = [col.split('.')[-1] for col in dataframe.columns]
    temp_df.append(dataframe)
    
for j in range(len(search_query_list)):
    data=temp_df[j]
    search_df.append(data)

search_df = pd.concat(search_df,sort=True)
search_df['categories'] = search_df.apply(get_category_type, axis=1)
search_df = pd.DataFrame(search_df[['name','categories','distance','lat','lng','id']])

/Users/abhigupta/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


### **Getting a frame of all categories**

In [65]:
pd.options.display.max_rows=None
pd.options.display.max_columns=None

sqldf('''select distinct ifnull(categories,name) categories from search_df order by 1''', locals())

,categories
0,Adult Boutique
1,American Restaurant
2,Arts & Crafts Store
3,Assisted Living
4,Bakery
5,Beer Bar
6,Beer Garden
7,Bishop Sundo Kim Memorial Church
8,Boutique
9,Buddhist Temple


In [66]:
df_venues = sqldf('''
select * from search_df 
where ifnull(categories,name) not in ('Road','Café') order by 1
''', locals())
df_venues.shape

(137, 6)

In [67]:
df_venues.head(10)

,name,categories,distance,lat,lng,id
0,117Park (117 파크),Steakhouse,859,37.571564,126.970742,562762c0498e66b48b709412
1,American Orientalism University,General College & University,869,37.572609,126.984699,551f44a4498e60974d9d3c59
2,Arirang Souvenir Shop,Souvenir Shop,972,37.572450,126.986570,5bd039c6364d970039ffc420
3,Artxstay Co-Working & Shop,Coworking Space,142,37.565425,126.978628,5b5fca3c031320002cfc1fbe
4,BARBER SHOP,Salon / Barbershop,729,37.563296,126.985369,51cd2fd3498ecf5841e193bb
5,BASF the chemical company,Office,781,37.560566,126.973927,4ffbfa20e4b0283dcdb0b3e1
6,BATTERY PARK (배터리파크),American Restaurant,825,37.560399,126.983277,5195f7b7498edc6527ea1208
7,Bain & Company (베인앤드컴퍼니코리아인크),Financial or Legal Service,827,37.560194,126.982877,4d40f274aae1b1f7bbe78bf5
8,Bethany Internasional Church Korea,Church,817,37.559361,126.979027,51fe068c498e09e91725d2b1
9,Bike Repair Shop by Bean Pole,Clothing Store,343,37.563846,126.979821,5261ee0a11d25d0c24b41772


### Creating a Folium Map using API

In [51]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, label in zip(df_venues.lat, df_venues.lng, df_venues.name):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill = True,
        fill_color='#3186cc',
        fill_opacity=0.7
    ).add_to(venues_map)
    
venues_map

### Adding cluste4r algorithm

In [52]:
kclusters = 3
kmeans = KMeans(n_clusters = kclusters, init ='k-means++')
kmeans.fit(df_venues[['lat','lng']])

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=3, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [53]:
df_venues['label'] = kmeans.fit_predict(df_venues[['lat','lng']])
df_venues.head(10)

,name,categories,distance,lat,lng,id,label
0,117Park (117 파크),Steakhouse,859,37.571564,126.970742,562762c0498e66b48b709412,0
1,American Orientalism University,General College & University,869,37.572609,126.984699,551f44a4498e60974d9d3c59,2
2,Arirang Souvenir Shop,Souvenir Shop,972,37.572450,126.986570,5bd039c6364d970039ffc420,2
3,Artxstay Co-Working & Shop,Coworking Space,142,37.565425,126.978628,5b5fca3c031320002cfc1fbe,1
4,BARBER SHOP,Salon / Barbershop,729,37.563296,126.985369,51cd2fd3498ecf5841e193bb,1
5,BASF the chemical company,Office,781,37.560566,126.973927,4ffbfa20e4b0283dcdb0b3e1,1
6,BATTERY PARK (배터리파크),American Restaurant,825,37.560399,126.983277,5195f7b7498edc6527ea1208,1
7,Bain & Company (베인앤드컴퍼니코리아인크),Financial or Legal Service,827,37.560194,126.982877,4d40f274aae1b1f7bbe78bf5,1
8,Bethany Internasional Church Korea,Church,817,37.559361,126.979027,51fe068c498e09e91725d2b1,1
9,Bike Repair Shop by Bean Pole,Clothing Store,343,37.563846,126.979821,5261ee0a11d25d0c24b41772,1


In [54]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=14)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

for lat, lng, cluster in zip(df_venues.lat, df_venues.lng, df_venues.label):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
#         popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.5
    ).add_to(venues_map)
    
venues_map

In [68]:
centers = kmeans.cluster_centers_
centers

array([[ 37.57054435, 126.97234437],
       [ 37.56188448, 126.9810365 ],
       [ 37.57039578, 126.98551057]])

### Querying for best matches

In [73]:
search_query_list = ['coffee']
radius =100
temp_df=[]
search_df=[]

for i in range(len(search_query_list)):
    url= 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
        CLIENT_ID, CLIENT_SECRET, centers[0][0], centers[0][1], VERSION, search_query_list[i], radius, LIMIT)
    results=requests.get(url).json()
    venues = results['response']['venues']
    dataframe = json_normalize(venues)
    dataframe.columns = [col.split('.')[-1] for col in dataframe.columns]
    temp_df.append(dataframe)
    
for j in range(len(search_query_list)):
    data=temp_df[j]
    search_df.append(data)

search_df = pd.concat(search_df,sort=True)
search_df['categories'] = search_df.apply(get_category_type, axis=1)
search_df = pd.DataFrame(search_df[['name','categories','distance','lat','lng','id']])
search_df

/Users/abhigupta/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  # This is added back by InteractiveShellApp.init_path()


,name,categories,distance,lat,lng,id
0,Get Out Coffee,Café,81,37.570167,126.973135,53193ee8498e1c63927a9f3f


In [70]:
search_query_list = ['coffee']
radius =100
temp_df=[]
search_df=[]

for i in range(len(search_query_list)):
    url= 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
        CLIENT_ID, CLIENT_SECRET, centers[1][0], centers[1][1], VERSION, search_query_list[i], radius, LIMIT)
    results=requests.get(url).json()
    venues = results['response']['venues']
    dataframe = json_normalize(venues)
    dataframe.columns = [col.split('.')[-1] for col in dataframe.columns]
    temp_df.append(dataframe)
    
for j in range(len(search_query_list)):
    data=temp_df[j]
    search_df.append(data)

search_df = pd.concat(search_df,sort=True)
search_df['categories'] = search_df.apply(get_category_type, axis=1)
search_df = pd.DataFrame(search_df[['name','categories','distance','lat','lng','id']])
search_df

/Users/abhigupta/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  # This is added back by InteractiveShellApp.init_path()


,name,categories,distance,lat,lng,id
0,Gloria Jean's Coffee,Coffee Shop,109,37.561694,126.982258,4e6822b4fa76aa094cef7250
1,Espreso coffee Rosebird 중앙클리닉점,Coffee Shop,82,37.561633,126.981921,4e1d0cdbb0fb93d2d05f6eb4
2,Barista Coffee,Coffee Shop,89,37.561371,126.980253,4bf0bce16f8aa5935140c23c
3,Paik's Coffee (빽다방),Coffee Shop,87,37.562422,126.981761,567b670f498ea638f1c5b8ec


In [71]:
search_query_list = ['coffee']
radius =100
temp_df=[]
search_df=[]

for i in range(len(search_query_list)):
    url= 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
        CLIENT_ID, CLIENT_SECRET, centers[2][0], centers[2][1], VERSION, search_query_list[i], radius, LIMIT)
    results=requests.get(url).json()
    venues = results['response']['venues']
    dataframe = json_normalize(venues)
    dataframe.columns = [col.split('.')[-1] for col in dataframe.columns]
    temp_df.append(dataframe)
    
for j in range(len(search_query_list)):
    data=temp_df[j]
    search_df.append(data)

search_df = pd.concat(search_df,sort=True)
search_df['categories'] = search_df.apply(get_category_type, axis=1)
search_df = pd.DataFrame(search_df[['name','categories','distance','lat','lng','id']])
search_df

/Users/abhigupta/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  # This is added back by InteractiveShellApp.init_path()


,name,categories,distance,lat,lng,id
0,The Coffee Bean & Tea Leaf,Coffee Shop,55,37.570004,126.985128,58e7159fe386e3088f77cc8a
1,EDIYA COFFEE,Coffee Shop,12,37.570435,126.985648,4da805fda86e68edb7d38d8a
2,EDIYA COFFEE,Coffee Shop,31,37.570454,126.985856,50176fc8e4b02c99f918848b
3,Mare coffee,Coffee Shop,107,37.569491,126.985085,4bdfa7956198c9b6741318ff
4,Angel-in-us Coffee,Coffee Shop,118,37.569390,126.985945,4d9196d856a3f04d03d4e641
5,That’s Coffee Jongro,Coffee Shop,140,37.569142,126.985639,5b440e39dd70c50039b1c859
6,Starbucks (스타벅스),Coffee Shop,99,37.569944,126.984535,4b48453ff964a520dd4a26e3


### The location for New coffee shop will be nearby the third center, which is 37.56991121, 126.97279993 at distance of 81m

# Conclusion

Hoping i could have used better APIs and available datasets around internet i could have predicted much better results. Although Foursquare and KMeans together did a great job as well.